In [11]:
#Funcion para lectura de datos .gpx
import gpxpy
import pandas as pd
import sqlite3
from geopy.distance import distance
from geopy.distance import geodesic

def read_gpx(file: str) -> pd.DataFrame:
    df = None
    points = []
    with open(file) as f:
        gpx = gpxpy.parse(f)

    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time':p.time,
                'latitude':p.latitude,
                'longitude':p.longitude,
                'elevation':p.elevation
            })
    df = pd.DataFrame.from_records(points)
    return df
    

In [12]:
#Datos
df1 = read_gpx('recovery.01-Mar-2022-1533.gpx')
df2 = read_gpx('recovery.05-Mar-2022.1025.gpx')
df3 = read_gpx('recovery.25-May-2022-0907.gpx')

In [3]:
#Separa Campos de fecha y hora
def separar_campos_fechas(df):

    df['date'] = pd.to_datetime(df['time'].dt.date)
    df['time'] = df['time'].dt.time
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    
    return df

In [4]:
separar_campos_fechas(df2)

,time,latitude,longitude,elevation,date,year,month,day
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04,2022,3,4
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04,2022,3,4
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04,2022,3,4
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04,2022,3,4
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04,2022,3,4
...,...,...,...,...,...,...,...,...
18588,15:25:40,6.255059,-75.590578,1481.257603,2022-03-05,2022,3,5
18589,15:25:42,6.255073,-75.590597,1480.845982,2022-03-05,2022,3,5
18590,15:25:44,6.255086,-75.590614,1480.847569,2022-03-05,2022,3,5
18591,15:25:46,6.255103,-75.590619,1480.186069,2022-03-05,2022,3,5


In [13]:
#Cargar datos a sqlite
def ingestar_sqlite(df, BD = 'Base.db', Tabla = 'Cordenadas'):
    conn = sqlite3.connect(BD)
    df.to_sql(Tabla, conn, if_exists='replace', index = False)
    df_consulta = pd.read_sql(f'SELECT * FROM {Tabla}', conn)
    conn.close()
    return df_consulta

In [14]:
ingestar_sqlite(df2)

,time,latitude,longitude,elevation
0,2022-03-04 00:04:34+00:00,6.180893,-75.577296,1589.639492
1,2022-03-04 00:04:35+00:00,6.180912,-75.577295,1588.165859
2,2022-03-04 00:04:43+00:00,6.180921,-75.577312,1591.435879
3,2022-03-04 00:04:46+00:00,6.180897,-75.577270,1587.116055
4,2022-03-04 00:04:48+00:00,6.180874,-75.577276,1586.772305
...,...,...,...,...
18588,2022-03-05 15:25:40+00:00,6.255059,-75.590578,1481.257603
18589,2022-03-05 15:25:42+00:00,6.255073,-75.590597,1480.845982
18590,2022-03-05 15:25:44+00:00,6.255086,-75.590614,1480.847569
18591,2022-03-05 15:25:46+00:00,6.255103,-75.590619,1480.186069


In [1]:
from haversine import haversine, Unit

# Coordenadas del punto A (latitud, longitud, elevación)
lat_A = 40.4168
lon_A = -3.7038
elev_A = 667

# Coordenadas del punto B (latitud, longitud, elevación)
lat_B = 52.5200
lon_B = 13.4050
elev_B = 34

# Calcular la distancia entre los puntos A y B
coord_A = (lat_A, lon_A)
coord_B = (lat_B, lon_B)
distancia_AB = haversine(coord_A, coord_B, Unit.METERS)

print(f"La distancia entre los puntos A y B es de {distancia_AB:.2f} metros.")

La distancia entre los puntos A y B es de 1869148.40 metros.
